# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to AIC and BIC!

## Summary

You will be able to:
- Build a linear regression model with interactions and polynomial features 
- Use AIC and BIC to select the best value for the regularization parameter 


## Let's get started!

Import all the necessary packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import scale,StandardScaler

from sklearn.datasets import load_boston

## Look at a baseline boston housing data model

- Import the Boston housing dataset 
- Split the data into target (`y`) and predictors (`X`) -- ensure these both are DataFrames 
- Scale all the predictors using `scale`. Convert these scaled features into a DataFrame 
- Build at a baseline model using *scaled variables* as predictors. Use 5-fold cross-validation (set `random_state` to 1) and use the $R^2$ score to evaluate the model 

In [10]:
boston = load_boston()
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [16]:
# print(boston.DESCR)

In [12]:
df = pd.DataFrame(boston['data'],columns=boston['feature_names'])
df['price'] = boston['target']
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [48]:
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


In [49]:
# Your code here
y = df['price'].copy()
X = df.drop(columns=['price']).copy()


X_train,X_test,y_train,y_test = train_test_split(X,y)
X_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
266,0.78570,20.0,3.97,0.0,0.6470,7.014,84.6,2.1329,5.0,264.0,13.0,384.07,14.79
4,0.06905,0.0,2.18,0.0,0.4580,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
401,14.23620,0.0,18.10,0.0,0.6930,6.343,100.0,1.5741,24.0,666.0,20.2,396.90,20.32
331,0.05023,35.0,6.06,0.0,0.4379,5.706,28.4,6.6407,1.0,304.0,16.9,394.02,12.43
407,11.95110,0.0,18.10,0.0,0.6590,5.608,100.0,1.2852,24.0,666.0,20.2,332.09,12.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,0.05497,0.0,5.19,0.0,0.5150,5.985,45.4,4.8122,5.0,224.0,20.2,396.90,9.74
98,0.08187,0.0,2.89,0.0,0.4450,7.820,36.9,3.4952,2.0,276.0,18.0,393.53,3.57
326,0.30347,0.0,7.38,0.0,0.4930,6.312,28.9,5.4159,5.0,287.0,19.6,396.90,6.15
144,2.77974,0.0,19.58,0.0,0.8710,4.903,97.8,1.3459,5.0,403.0,14.7,396.90,29.29


In [109]:
scaler = StandardScaler()
X_train_sca = pd.DataFrame(scaler.fit_transform(X_train),
                           columns=X_train.columns, index=X_train.index)
X_test_sca = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns,
                         index=X_test.index)

X_scaled = pd.concat([X_train_sca,X_test_sca],axis=0)
X_scaled

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
266,-0.323360,0.413924,-1.063313,-0.265747,0.794330,1.035078,0.584656,-0.778400,-0.502046,-0.842843,-2.610287,0.311818,0.339924
4,-0.415993,-0.469393,-1.327970,-0.265747,-0.824957,1.221922,-0.495534,1.108219,-0.735799,-1.095505,0.083161,0.445110,-1.022927
401,1.415224,-0.469393,1.025851,-0.265747,1.188442,0.092430,1.131857,-1.046703,1.718612,1.575497,0.791963,0.445110,1.136602
331,-0.418425,1.076411,-0.754300,-0.265747,-0.997166,-0.802453,-1.412274,1.385981,-0.969553,-0.602212,-0.767401,0.415189,-0.000068
407,1.119856,-0.469393,1.025851,-0.265747,0.897142,-0.940127,1.131857,-1.185416,1.718612,1.575497,0.791963,-0.228208,-0.043288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,-0.420977,1.959728,-1.091405,-0.265747,-0.602198,0.838401,-1.422934,1.301812,-0.502046,-0.205171,-0.436627,0.352335,-1.126654
17,-0.323554,-0.469393,-0.446766,-0.265747,-0.139544,-0.403478,0.481612,0.241900,-0.618922,-0.584165,1.169991,0.339661,0.322636
420,1.008216,-0.469393,1.025851,-0.265747,1.402633,0.187959,1.131857,-0.909959,1.718612,1.575497,0.791963,-0.366798,0.373059
136,-0.383214,-0.469393,1.586214,-0.265747,0.597274,-0.470911,0.900896,-0.858103,-0.618922,0.197885,1.264498,0.251353,0.643900


In [100]:
## Shuffle=False
crossval = KFold(n_splits=5,random_state=1,shuffle=False)

linreg = LinearRegression()
scores = cross_val_score(linreg,X,y=y,cv=crossval,scoring='r2')
scores.mean()

0.3532759243958823

In [106]:
## Shuffle=True
crossval = KFold(n_splits=5,random_state=1,shuffle=True)

linreg = LinearRegression()
scores = cross_val_score(linreg,X,y=y,cv=crossval,scoring='r2')
scores.mean()

0.7176778617934929

## Include interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Next, evaluate that model using 5-fold cross-validation and store the $R^2$ to compare it with the baseline model.

Print the 7 most important interactions.

In [112]:
combs[0]

('CRIM', 'ZN')

In [110]:
# Your code here
from itertools import combinations,product
combs = list(combinations(X.columns,2))
interactions = [['Interaction','R2']]

data = X_scaled.copy()
for (col1,col2) in combs:
    ixn_name = f"{col1}_{col2}"
    data[ixn_name] = data[col1] * data[col2]
    score = cross_val_score(linreg,data,y,cv=crossval,scoring='r2').mean()
    interactions.append([ixn_name, score])
    
res_df = pd.DataFrame(interactions[1:],columns=interactions[0])
res_df.sort_values('R2',ascending=False)

,Interaction,R2
0,CRIM_ZN,-0.075209
1,CRIM_INDUS,-0.085453
2,CRIM_CHAS,-0.086579
3,CRIM_NOX,-0.088159
5,CRIM_AGE,-0.088396
...,...,...
68,RAD_TAX,-0.555334
74,TAX_LSTAT,-0.558439
75,PTRATIO_B,-0.573493
76,PTRATIO_LSTAT,-0.639604


In [111]:
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,RAD_TAX,RAD_PTRATIO,RAD_B,RAD_LSTAT,TAX_PTRATIO,TAX_B,TAX_LSTAT,PTRATIO_B,PTRATIO_LSTAT,B_LSTAT
266,-0.323360,0.413924,-1.063313,-0.265747,0.794330,1.035078,0.584656,-0.778400,-0.502046,-0.842843,...,0.423145,1.310483,-0.156547,-0.170657,2.200061,-0.262813,-0.286503,-0.813934,-0.887299,0.105994
4,-0.415993,-0.469393,-1.327970,-0.265747,-0.824957,1.221922,-0.495534,1.108219,-0.735799,-1.095505,...,0.806072,-0.061190,-0.327511,0.752669,-0.091103,-0.487620,1.120622,0.037016,-0.085068,-0.455315
401,1.415224,-0.469393,1.025851,-0.265747,1.188442,0.092430,1.131857,-1.046703,1.718612,1.575497,...,2.707668,1.361077,0.764971,1.953377,1.247735,0.701269,1.790712,0.352511,0.900147,0.505913
331,-0.418425,1.076411,-0.754300,-0.265747,-0.997166,-0.802453,-1.412274,1.385981,-0.969553,-0.602212,...,0.583876,0.744036,-0.402548,0.000066,0.462138,-0.250032,0.000041,-0.318617,0.000053,-0.000028
407,1.119856,-0.469393,1.025851,-0.265747,0.897142,-0.940127,1.131857,-1.185416,1.718612,1.575497,...,2.707668,1.361077,-0.392200,-0.074395,1.247735,-0.359540,-0.068200,-0.180732,-0.034282,0.009879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,-0.420977,1.959728,-1.091405,-0.265747,-0.602198,0.838401,-1.422934,1.301812,-0.502046,-0.205171,...,0.103005,0.219207,-0.176888,0.565632,0.089583,-0.072289,0.231157,-0.153839,0.491928,-0.396960
17,-0.323554,-0.469393,-0.446766,-0.265747,-0.139544,-0.403478,0.481612,0.241900,-0.618922,-0.584165,...,0.361553,-0.724134,-0.210223,-0.199687,-0.683467,-0.198418,-0.188473,0.397400,0.377482,0.109587
420,1.008216,-0.469393,1.025851,-0.265747,1.402633,0.187959,1.131857,-0.909959,1.718612,1.575497,...,2.707668,1.361077,-0.630384,0.641144,1.247735,-0.577889,0.587753,-0.290491,0.295449,-0.136837
136,-0.383214,-0.469393,1.586214,-0.265747,0.597274,-0.470911,0.900896,-0.858103,-0.618922,0.197885,...,-0.122476,-0.782626,-0.155568,-0.398524,0.250226,0.049739,0.127419,0.317836,0.814211,0.161846


Write code to include the 7 most important interactions in your data set by adding 7 columns. Name the columns "var1_var2" with var1 and var2 the two variables in the interaction.

In [ ]:
# Your code here

## Include polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of 4, the particular column is raised to the power of 2 and 3 as well in other terms. We only want to include "pure" polynomials, so make sure no interactions are included. We want the result to return a list that contain tuples of the form:

`(var_name, degree, R2)`, so eg. `('DIS', 3, 0.732)` 

In [ ]:
# Your code here

For each variable, print out the maximum R2 possible when including Polynomials.

In [ ]:
# Your code here

Which two variables seem to benefit most from adding polynomial terms?

Add Polynomials for the two features that seem to benefit the most, as in have the best R squared compared to the baseline model. For each of the two features, raise to the Polynomial that generates the best result. Make sure to start from the data set `df_inter` so the final data set has both interactions and polynomials in the model.

In [ ]:
# Your code here

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [ ]:
# Your code here

## Full model R-squared

Check out the R-squared of the full model.

In [ ]:
# Your code here

## Find the best Lasso regularization parameter

You learned that when using Lasso regularization, your coefficients shrink to 0 when using a higher regularization parameter. Now the question is which value we should choose for the regularization parameter. 

This is where the AIC and BIC come in handy! We'll use both criteria in what follows and perform cross-validation to select an optimal value of the regularization parameter $alpha$ of the Lasso estimator.

Read the page here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html and create a similar plot as the first one listed on the page. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC

In [ ]:
# Your code here 


## Analyze the final result

Finally, use the best value for the regularization parameter according to AIC and BIC, and compare R-squared and MSE using train-test split. Compare with the baseline model.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

In [ ]:
# Split X_scaled and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = None

# Code for baseline model
linreg_all = None


# Print R2 and MSE


In [ ]:
# Split df_inter and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = None

# Code for lasso with alpha from AIC
lasso = None


# Print R2 and MSE


In [ ]:
# Code for lasso with alpha from BIC
lasso = None


# Print R2 and MSE


## Level up (Optional)

### Create a Lasso path

From this section, you know that when using lasso, more parameters shrink to zero as your regularization parameter goes up. In Scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for subset selection
This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Boston housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to create better linear models and how to use AIC and BIC for both feature selection and to optimize your regularization parameter when performing Ridge and Lasso. 